In [1]:

# # Install the PyDrive wrapper & import libraries.
# # This only needs to be done once in a notebook.
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once in a notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [2]:
# #https://drive.google.com/open?id=1HpgQosA-EmVcspFlNNQB45X6d6wk5rvj
# file_id = '1HpgQosA-EmVcspFlNNQB45X6d6wk5rvj' 
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('train.csv')

In [3]:
# #https://drive.google.com/open?id=1l1HSiofIlQBE4fTfr0AMVWQUgvPLb9uH
# file_id = '1l1HSiofIlQBE4fTfr0AMVWQUgvPLb9uH' 
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('test.csv')

In [4]:
# #https://drive.google.com/open?id=1Xa1dciuSpB_w4TVZYdi4IJluO8Wwg_pR
# file_id = '1Xa1dciuSpB_w4TVZYdi4IJluO8Wwg_pR' 
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('sample_submission.csv')

In [5]:
# !ls

### Load Packages

In [6]:
import numpy as np
import pandas as pd
from scipy.stats import mode
from sklearn import metrics, preprocessing, model_selection
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import lightgbm as lgb
import matplotlib.pyplot as plt

%matplotlib inline

pd.options.display.max_columns = 100
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import warnings
warnings.filterwarnings("ignore")

## Read the datasets

In [7]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission_df = pd.read_csv('sample_submission.csv')

In [8]:
print(f'Number of samples in train: {train_df.shape[0]}')
print(f'Number of columns in train: {train_df.shape[1]}')
for col in train_df.columns:
    if train_df[col].isnull().any():
        print(col, train_df[col].isnull().sum())

Number of samples in train: 341424
Number of columns in train: 24
season_holidayed_code 114
state_code_residence 4764


In [9]:
train_df.head(3)

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,7.706428
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,6.662563
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,7.871602


In [10]:
print(f'Number of samples in test: {test_df.shape[0]}')
print(f'Number of columns in test: {test_df.shape[1]}')
for col in test_df.columns:
    if test_df[col].isnull().any():
        print(col, test_df[col].isnull().sum())

Number of samples in test: 146765
Number of columns in test: 23
season_holidayed_code 35
state_code_residence 2260


In [11]:
train_df.dtypes

reservation_id                         object
booking_date                           object
checkin_date                           object
checkout_date                          object
channel_code                            int64
main_product_code                       int64
numberofadults                          int64
numberofchildren                        int64
persontravellingid                      int64
resort_region_code                      int64
resort_type_code                        int64
room_type_booked_code                   int64
roomnights                              int64
season_holidayed_code                 float64
state_code_residence                  float64
state_code_resort                       int64
total_pax                               int64
member_age_buckets                     object
booking_type_code                       int64
memberid                               object
cluster_code                           object
reservationstatusid_code          

In [12]:
# Check if there is any shift between the users in train and test
## Get the unique users in trian and test set
uniq_users_train = np.array(train_df['memberid'].unique().tolist())
uniq_users_train

print('The total number of users from train are :', uniq_users_train.size)

uniq_users_test = np.array(test_df['memberid'].unique().tolist())
uniq_users_test
print('The total number of  users from test are :', uniq_users_test.size)

## Get the common number of users between train and test
common_users = (np.intersect1d(uniq_users_train, uniq_users_test))
print('The total number of common users are :',common_users.size)

The total number of users from train are : 101327
The total number of  users from test are : 43496
The total number of common users are : 0


In [13]:
# Check if there is any shift between the users in train and test
## Get the unique users in trian and test set
uniq_users_train = np.array(train_df['reservation_id'].unique().tolist())
uniq_users_train

print('The total number of users from train are :', uniq_users_train.size)

uniq_users_test = np.array(test_df['reservation_id'].unique().tolist())
uniq_users_test
print('The total number of  users from test are :', uniq_users_test.size)

## Get the common number of users between train and test
common_users = (np.intersect1d(uniq_users_train, uniq_users_test))
print('The total number of common users are :',common_users.size)

The total number of users from train are : 341424
The total number of  users from test are : 146765
The total number of common users are : 0


In [14]:
print(train_df.shape)
print(test_df.shape)

(341424, 24)
(146765, 23)


In [15]:
train_df.dtypes

reservation_id                         object
booking_date                           object
checkin_date                           object
checkout_date                          object
channel_code                            int64
main_product_code                       int64
numberofadults                          int64
numberofchildren                        int64
persontravellingid                      int64
resort_region_code                      int64
resort_type_code                        int64
room_type_booked_code                   int64
roomnights                              int64
season_holidayed_code                 float64
state_code_residence                  float64
state_code_resort                       int64
total_pax                               int64
member_age_buckets                     object
booking_type_code                       int64
memberid                               object
cluster_code                           object
reservationstatusid_code          

In [16]:
cols = ['member_age_buckets','memberid', 'cluster_code', 'reservationstatusid_code', 'resort_id'  ]
for col in cols:
    if train_df[col].dtype==object:
        print(col)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
        train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
        test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))

member_age_buckets
memberid
cluster_code
reservationstatusid_code
resort_id


In [17]:
train_df.dtypes

reservation_id                         object
booking_date                           object
checkin_date                           object
checkout_date                          object
channel_code                            int64
main_product_code                       int64
numberofadults                          int64
numberofchildren                        int64
persontravellingid                      int64
resort_region_code                      int64
resort_type_code                        int64
room_type_booked_code                   int64
roomnights                              int64
season_holidayed_code                 float64
state_code_residence                  float64
state_code_resort                       int64
total_pax                               int64
member_age_buckets                      int64
booking_type_code                       int64
memberid                                int64
cluster_code                            int64
reservationstatusid_code          

In [18]:
train_df.head(2)

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,5,1,34521,5,2,7,7.706428
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,5,1,34521,5,0,1,6.662563


In [19]:
# append train and test data
mergedata = train_df.append(test_df)
testcount = len(test_df)
count = len(mergedata)-testcount
print(count)

341424


In [20]:
print(mergedata.shape)

(488189, 24)


In [21]:
mergedata.tail(3)

,amount_spent_per_room_night_scaled,booking_date,booking_type_code,channel_code,checkin_date,checkout_date,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservation_id,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax
146762,NaN,23/02/19,1,1,24/02/19,26/02/19,5,1,5,139451,5,0,45,ed7c7ab2ff18b02e5bb10a99e6883d1529171fa330029a...,0,5,1,1,2,4,4.0,3.0,6,3
146763,NaN,26/02/19,1,1,01/03/19,02/03/19,0,7,8,117240,2,2,4995,381d17a0a2d8912650092cbd5e0b16a4f93b6d0fd7d174...,0,2,3,3,3,2,3.0,5.0,3,4
146764,NaN,27/02/19,2,2,01/03/19,02/03/19,2,4,1,16820,2,0,47,4cc41366b74c34ff64babc84eab1da7b45810056785fb8...,0,26,2,1,4,1,3.0,8.0,7,2


### Missing values check

In [22]:
mergedata.isnull().sum()

amount_spent_per_room_night_scaled    146765
booking_date                               0
booking_type_code                          0
channel_code                               0
checkin_date                               0
checkout_date                              0
cluster_code                               0
main_product_code                          0
member_age_buckets                         0
memberid                                   0
numberofadults                             0
numberofchildren                           0
persontravellingid                         0
reservation_id                             0
reservationstatusid_code                   0
resort_id                                  0
resort_region_code                         0
resort_type_code                           0
room_type_booked_code                      0
roomnights                                 0
season_holidayed_code                    149
state_code_residence                    7024
state_code

#### Impute missing values

In [23]:

# season_holidayed_code                    149
# state_code_residence                    7024

mergedata['season_holidayed_code'].fillna(mode(mergedata['season_holidayed_code']).mode[0], inplace=True)
mergedata['state_code_residence'].fillna(mode(mergedata['state_code_residence']).mode[0], inplace=True)
# roomnights
mergedata['roomnights'].replace(to_replace =-45, value =45,inplace= True) 
mergedata['roomnights'].replace(to_replace =0, value =5,inplace= True) 
mergedata['total_pax'].replace(to_replace =0, value =5,inplace= True) 
mergedata['total_pax'].replace(to_replace =0, value =5,inplace= True)
mergedata['numberofadults'].replace(to_replace =0, value =1,inplace= True)
mergedata['numberofchildren'].replace(to_replace =0, value =1,inplace= True) 

In [24]:
dates_df = mergedata[['booking_date', 'checkin_date',	'checkout_date']]

In [25]:
dates_df.head(3)

,booking_date,checkin_date,checkout_date
0,05/04/18,05/04/18,06/04/18
1,23/01/15,11/04/15,16/04/15
2,28/01/15,01/02/15,05/02/15


In [26]:
dates_df.dtypes

booking_date     object
checkin_date     object
checkout_date    object
dtype: object

### Feature Engineering

In [27]:
dates_df[['booking_day','booking_month','booking_year']] = dates_df.booking_date.str.split("/",expand=True,).astype(int)

dates_df[['checkin_day','checkin_month','checkin_year']] = dates_df.checkin_date.str.split("/",expand=True,).astype(int)

dates_df[['checkout_day','checkout_month','checkout_year']] = dates_df.checkout_date.str.split("/",expand=True,).astype(int)
dates_df.head(5)

,booking_date,checkin_date,checkout_date,booking_day,booking_month,booking_year,checkin_day,checkin_month,checkin_year,checkout_day,checkout_month,checkout_year
0,05/04/18,05/04/18,06/04/18,5,4,18,5,4,18,6,4,18
1,23/01/15,11/04/15,16/04/15,23,1,15,11,4,15,16,4,15
2,28/01/15,01/02/15,05/02/15,28,1,15,1,2,15,5,2,15
3,02/05/15,11/06/15,16/06/15,2,5,15,11,6,15,16,6,15
4,02/09/15,14/12/15,19/12/15,2,9,15,14,12,15,19,12,15


In [28]:
dates_df['booking_year']=dates_df['booking_year'].apply(lambda x: {12: 2012, 13:2013, 14: 2014, 15: 2015, 16:2016,
                                     17:2017, 18:2018, 19:2019}.get(x,x))
dates_df['checkin_year']=dates_df['checkin_year'].apply(lambda x: {12: 2012, 13:2013, 14: 2014, 15: 2015, 16:2016,
                                     17:2017, 18:2018, 19:2019}.get(x,x))
dates_df['checkout_year']=dates_df['checkout_year'].apply(lambda x: {12: 2012, 13:2013, 14: 2014, 15: 2015, 16:2016,
                                     17:2017, 18:2018, 19:2019}.get(x,x))

dates_df['checkin_year']= dates_df['checkin_year'].replace(2012, 2018)
dates_df['checkout_year']= dates_df['checkout_year'].replace(2012, 2018)

In [29]:
dates_df['new_booking_date'] = pd.to_datetime(dict(year=dates_df.booking_year, month=dates_df.booking_month, day=dates_df.booking_day))
dates_df['new_checkin_date'] = pd.to_datetime(dict(year=dates_df.checkin_year, month=dates_df.checkin_month, day=dates_df.checkin_day))
dates_df['new_checkout_date'] = pd.to_datetime(dict(year=dates_df.checkout_year, month=dates_df.checkout_month, day=dates_df.checkout_day))




In [30]:
dates_df.head(3)

,booking_date,checkin_date,checkout_date,booking_day,booking_month,booking_year,checkin_day,checkin_month,checkin_year,checkout_day,checkout_month,checkout_year,new_booking_date,new_checkin_date,new_checkout_date
0,05/04/18,05/04/18,06/04/18,5,4,2018,5,4,2018,6,4,2018,2018-04-05,2018-04-05,2018-04-06
1,23/01/15,11/04/15,16/04/15,23,1,2015,11,4,2015,16,4,2015,2015-01-23,2015-04-11,2015-04-16
2,28/01/15,01/02/15,05/02/15,28,1,2015,1,2,2015,5,2,2015,2015-01-28,2015-02-01,2015-02-05


In [31]:
dates_df.drop(['booking_date', 'checkin_date', 'checkout_date'],axis=1,inplace=True)

In [32]:
dates_df.dtypes

booking_day                   int32
booking_month                 int32
booking_year                  int64
checkin_day                   int32
checkin_month                 int32
checkin_year                  int64
checkout_day                  int32
checkout_month                int32
checkout_year                 int64
new_booking_date     datetime64[ns]
new_checkin_date     datetime64[ns]
new_checkout_date    datetime64[ns]
dtype: object

In [33]:
dates_df['booking_dayofweek'] = dates_df['new_booking_date'].dt.dayofweek
dates_df['booking_is_weekend'] = dates_df['booking_dayofweek'].apply(lambda x: 1 if x > 5 else 0)

dates_df['checkin_dayofweek'] = dates_df['new_checkin_date'].dt.dayofweek
dates_df['checkin_is_weekend'] = dates_df['checkin_dayofweek'].apply(lambda x: 1 if x > 5 else 0)

dates_df['checkout_dayofweek'] = dates_df['new_checkout_date'].dt.dayofweek
dates_df['checkout_is_weekend'] = dates_df['checkout_dayofweek'].apply(lambda x: 1 if x > 5 else 0)


dates_df['diff_book_checkin_days'] = (dates_df['new_checkin_date'] - dates_df['new_booking_date']).dt.days
dates_df['diff_checkout_checkin_days'] = (dates_df['new_checkout_date'] - dates_df['new_checkin_date']).dt.days
dates_df['diff_book_checkout_days'] = (dates_df['new_checkout_date'] - dates_df['new_booking_date']).dt.days



dates_df['diff_book_checkin_days'] = dates_df['diff_book_checkin_days'].apply(lambda x: abs(x))
dates_df['diff_checkout_checkin_days'] = dates_df['diff_checkout_checkin_days'].apply(lambda x: abs(x))
dates_df['diff_book_checkout_days'] = dates_df['diff_book_checkout_days'].apply(lambda x: abs(x))



In [34]:
FinalData = pd.concat([mergedata, 
                       dates_df], axis=1)
FinalData = FinalData.loc[:,~FinalData.columns.duplicated()]

#remove some  columns
FinalData.drop([
    'booking_date','checkin_date', 'checkout_date'            
               ],axis=1,inplace=True)

In [35]:
#add adults + children
FinalData['numberof_adults_children'] = FinalData['numberofadults'] + FinalData['numberofchildren']

gdf = FinalData.groupby(["memberid"])["diff_checkout_checkin_days"].agg(["min", "max", "mean", "median"]).reset_index()
gdf.columns = ["memberid", "memberdays_min", "memberdays_max", "memberdays_mean", "memberdays_median"]
FinalData = FinalData.merge(gdf, on=["memberid"], how="left")

In [36]:
gdf = FinalData.groupby(["memberid"])["diff_book_checkin_days"].agg(["min", "max", "mean", "median"]).reset_index()
gdf.columns = ["memberid", "memberdaysbook_min", "memberdaysbook_max",
               "memberdaysbook_mean","memberdaysbook_median" ]
FinalData = FinalData.merge(gdf, on=["memberid"], how="left")


gdf = FinalData.groupby(["memberid"])["diff_book_checkout_days"].agg(["min", "max", "mean", "median"]).reset_index()
gdf.columns = ["memberid", "memberdaysbook_chkout_min", "memberdaysbook_chkout_max",
               "memberdaysbook_chkout_mean","memberdaysbook_chkout_median" ]
FinalData = FinalData.merge(gdf, on=["memberid"], how="left")

In [37]:
for col1, col2 in [["roomnights", "total_pax"],
                   ["numberofchildren", "numberofadults"],
                   ["numberofchildren", "total_pax"],
                   ["numberofadults", "total_pax"]
                  ]:
    FinalData[col1+"_ratio_"+col2] = FinalData[col1] / FinalData[col2]
    
    

gdf = FinalData.groupby(["memberid"])["roomnights"].agg(["min", "max", "mean", "median"]).reset_index()
gdf.columns = ["memberid", "mem_room_nights_min", "mem_room_nights_max", "mem_room_nights_mean" ,
               "mem_room_nights_median"]           
FinalData = FinalData.merge(gdf, on=["memberid"], how="left")


gdf = FinalData.groupby(["memberid"])["numberof_adults_children"].agg(["min", "max", "mean","median"]).reset_index()
gdf.columns = ["memberid", "no_ad_c_min", "no_ad_cs_max", "no_ad_c_mean" , "no_ad_c_median"]               
FinalData = FinalData.merge(gdf, on=["memberid"], how="left")


  
gdf = FinalData.groupby(["memberid"])["total_pax"].agg(["min", "max", "mean", "median"]).reset_index()
gdf.columns = ["memberid", "totalpax_min", "totalpax_max", "totalpax_mean", "totalpax_median"]
FinalData = FinalData.merge(gdf, on=["memberid"], how="left")

#resort id features

gdf = FinalData.groupby(["resort_id"])["diff_book_checkin_days"].agg(["min", "max", "mean", "median"]).reset_index()
gdf.columns = ["resort_id", "resort_id_book_min", "resort_id_book_max",
               "resort_id_book_mean","resort_id_book_median" ]
FinalData = FinalData.merge(gdf, on=["resort_id"], how="left")

gdf = FinalData.groupby(["resort_id"])["diff_checkout_checkin_days"].agg(["min", "max", "mean", "median"]).reset_index()
gdf.columns = ["resort_id", "resort_id_checkout_checkin_min", "resort_id_checkout_checkin_max",
               "resort_id_checkout_checkin_mean","resort_id_checkout_checkin_median" ]
FinalData = FinalData.merge(gdf, on=["resort_id"], how="left")

gdf = FinalData.groupby(["resort_id"])["diff_book_checkout_days"].agg(["min", "max", "mean", "median"]).reset_index()
gdf.columns = ["resort_id", "resort_id_book_checkout__min", "resort_id_book_checkout_max",
               "resort_id_book_checkout_mean","resort_id_book_checkout_median" ]
FinalData = FinalData.merge(gdf, on=["resort_id"], how="left")


gdf = FinalData.groupby(["resort_id"])["roomnights"].agg(["min", "max", "mean", "median"]).reset_index()
gdf.columns = ["resort_id", "res_room_nights_min", "res_room_nights_max", "res_room_nights_mean" ,
               "res_room_nights_median"]
FinalData = FinalData.merge(gdf, on=["resort_id"], how="left")

gdf = FinalData.groupby(["resort_id"])["numberof_adults_children"].agg(["min", "max", "mean","median"]).reset_index()
gdf.columns = ["resort_id", "res_ad_c_min", "res_ad_cs_max", "res_ad_c_mean" , "res_ad_c_median"]               
FinalData = FinalData.merge(gdf, on=["resort_id"], how="left")
  
gdf = FinalData.groupby(["resort_id"])["total_pax"].agg(["min", "max", "mean", "median"]).reset_index()
gdf.columns = ["resort_id", "res_totalpax_min", "res_totalpax_max", "res_totalpax_mean", "res_totalpax_median"]
FinalData = FinalData.merge(gdf, on=["resort_id"], how="left")

for col in ['resort_region_code', 'resort_type_code', 'channel_code',	'main_product_code',
            'persontravellingid','room_type_booked_code', 'season_holidayed_code',
            'booking_type_code',
            'state_code_residence', 'state_code_resort', 'member_age_buckets']:
    if not isinstance(col, list):
        col = [col]
    col_name = "_".join(col)
    gdf = FinalData.groupby(col)["diff_book_checkin_days"].agg(["min", "max", "mean", "median"]).reset_index()
    gdf.columns = col + [col_name+"_diff_book_checkin_days_min",col_name+"_diff_book_checkin_days_max", col_name+"_diff_book_checkin_days_mean", col_name+"_diff_book_checkin_days_median"]
    FinalData = FinalData.merge(gdf, on=col, how="left")
    
   
    gdf = FinalData.groupby(col)["diff_checkout_checkin_days"].agg(["min", "max", "mean", "median"]).reset_index()
    gdf.columns = col + [col_name+"_diff_checkout_checkin_days_min",col_name+"_diff_checkout_checkin_days_max", col_name+"_diff_checkout_checkin_days_mean", col_name+"_diff_checkout_checkin_days_median"]
    FinalData = FinalData.merge(gdf, on=col, how="left")
    
    gdf = FinalData.groupby(col)["diff_book_checkout_days"].agg(["min", "max", "mean", "median"]).reset_index()
    gdf.columns = col + [col_name+"_diff_book_checkout_days_min",col_name+"_diff_book_checkout_days_max", col_name+"_diff_checkout_checkin_days_mean", col_name+"_diff_checkout_checkin_days_median"]
    FinalData = FinalData.merge(gdf, on=col, how="left")
    
    gdf = FinalData.groupby(col)["numberof_adults_children"].agg(["min", "max", "mean", "median"]).reset_index()
    gdf.columns = col + [col_name+"_numberof_adults_children_min",col_name+"_numberof_adults_children_max", col_name+"_numberof_adults_children_mean", col_name+"_numberof_adults_children_median"]
    FinalData = FinalData.merge(gdf, on=col, how="left")
    
    gdf = FinalData.groupby(col)["roomnights"].agg(["min", "max", "mean", "median"]).reset_index()
    gdf.columns = col + [col_name+"_roomnights_min",col_name+"_roomnights_max", col_name+"_roomnights_mean", col_name+"_roomnights_median"]
    FinalData = FinalData.merge(gdf, on=col, how="left")
    
    gdf = FinalData.groupby(col)["total_pax"].agg(["min", "max", "mean", "median"]).reset_index()
    gdf.columns = col + [col_name+"_total_pax_min",col_name+"_total_pax_max", col_name+"_total_pax_mean", col_name+"_total_pax_median"]
    FinalData = FinalData.merge(gdf, on=col, how="left")
 

    
## Count features
for col in ["memberid", 'resort_id', 'resort_type_code', 'roomnights', 
            'state_code_residence', 'state_code_resort', 'persontravellingid',            
            'channel_code',	'main_product_code',  
            'cluster_code', 'room_type_booked_code','season_holidayed_code','booking_day',
            'checkin_day','checkout_day',
             'member_age_buckets',            
            ["memberid", 'resort_id'], 
            ['resort_region_code', 'resort_type_code'], 
           ["state_code_residence", "cluster_code"],
            ["state_code_residence", "resort_id"],
            

           ]:
    if not isinstance(col, list):
        col = [col]
    col_name = "_".join(col)
    gdf = FinalData.groupby(col)["reservation_id"].count().reset_index()
    gdf.columns = col + [col_name+"_count"]
    FinalData = FinalData.merge(gdf, on=col, how="left")
   

date_cols = ['booking_year',	'booking_month',	'booking_day',	'booking_dayofweek',
'checkin_year',	'checkin_month',	'checkin_day',	'checkin_dayofweek',
'checkout_year',	'checkout_month',	'checkout_day',	'checkout_dayofweek']

le = preprocessing.LabelEncoder()
lbl_enc_dates = FinalData[date_cols]

lbl_enc_dates = lbl_enc_dates.apply(le.fit_transform) 


FinalData.drop(date_cols,axis=1,inplace=True)

FinalData = pd.concat([FinalData,lbl_enc_dates], axis=1)
FinalData = FinalData.loc[:,~FinalData.columns.duplicated()]


#remove some  columns
FinalData.drop([
    'new_booking_date','new_checkin_date', 'new_checkout_date',
                'numberofadults',  'numberofchildren'
               ],axis=1,inplace=True)

       
# FinalData= FinalData.drop(cols,axis=1)
train = FinalData[:count]
test = FinalData[count:]


In [38]:
print(FinalData.shape)
print(train.shape)
print(test.shape)

(488189, 374)
(341424, 374)
(146765, 374)


In [39]:
train_df = train.copy()
test_df = test.copy()

In [40]:
train_df.tail(3)

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code,main_product_code,member_age_buckets,memberid,persontravellingid,reservation_id,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,booking_is_weekend,checkin_is_weekend,checkout_is_weekend,diff_book_checkin_days,diff_checkout_checkin_days,diff_book_checkout_days,numberof_adults_children,memberdays_min,memberdays_max,memberdays_mean,memberdays_median,memberdaysbook_min,memberdaysbook_max,memberdaysbook_mean,memberdaysbook_median,memberdaysbook_chkout_min,memberdaysbook_chkout_max,memberdaysbook_chkout_mean,memberdaysbook_chkout_median,roomnights_ratio_total_pax,numberofchildren_ratio_numberofadults,numberofchildren_ratio_total_pax,numberofadults_ratio_total_pax,mem_room_nights_min,mem_room_nights_max,mem_room_nights_mean,mem_room_nights_median,no_ad_c_min,no_ad_cs_max,no_ad_c_mean,no_ad_c_median,...,member_age_buckets_diff_checkout_checkin_days_mean_x,member_age_buckets_diff_checkout_checkin_days_median_x,member_age_buckets_diff_book_checkout_days_min,member_age_buckets_diff_book_checkout_days_max,member_age_buckets_diff_checkout_checkin_days_mean_y,member_age_buckets_diff_checkout_checkin_days_median_y,member_age_buckets_numberof_adults_children_min,member_age_buckets_numberof_adults_children_max,member_age_buckets_numberof_adults_children_mean,member_age_buckets_numberof_adults_children_median,member_age_buckets_roomnights_min,member_age_buckets_roomnights_max,member_age_buckets_roomnights_mean,member_age_buckets_roomnights_median,member_age_buckets_total_pax_min,member_age_buckets_total_pax_max,member_age_buckets_total_pax_mean,member_age_buckets_total_pax_median,memberid_count,resort_id_count,resort_type_code_count,roomnights_count,state_code_residence_count,state_code_resort_count,persontravellingid_count,channel_code_count,main_product_code_count,cluster_code_count,room_type_booked_code_count,season_holidayed_code_count,booking_day_count,checkin_day_count,checkout_day_count,member_age_buckets_count,memberid_resort_id_count,resort_region_code_resort_type_code_count,state_code_residence_cluster_code_count,state_code_residence_resort_id_count,booking_year,booking_month,booking_day,booking_dayofweek,checkin_year,checkin_month,checkin_day,checkin_dayofweek,checkout_year,checkout_month,checkout_day,checkout_dayofweek
341421,8.381939,1,3,5,3,7,137827,45,18a5668c302c321dc2d42a48923ceff97af633c66d72ee...,0,6,1,1,3,2,4.0,10.0,4,3,0,0,0,0,1,1,4,1,1,1.0,1.0,0,0,0.0,0.0,1,1,1.0,1.0,0.666667,0.333333,0.333333,1.000000,2,2,2.0,2.0,4,4,4.0,4.0,...,2.574337,2,1,127,51.051296,40,2,33,4.397033,4,1,39,4.000567,3,1,24,3.007153,3,1,2030,220256,121780,13468,21168,382100,182950,123267,172770,196438,99276,12559,15969,16839,54663,1,141419,4328,135,5,1,26,2,4,1,26,2,4,1,27,3
341422,8.856035,1,1,3,3,1,100185,47,1cbe1611a122023d7e994b4cf51fcf28ce44acdbcabbfb...,0,10,2,2,2,2,3.0,8.0,2,3,0,0,0,1,1,2,6,1,1,1.0,1.0,1,1,1.0,1.0,2,2,2.0,2.0,0.666667,0.200000,0.333333,1.666667,2,2,2.0,2.0,6,6,6.0,6.0,...,2.242163,2,1,126,42.941800,27,2,25,4.138824,3,1,28,3.287663,3,1,11,3.108781,3,1,27224,106308,121780,199403,80128,45062,230506,123267,80128,162447,169640,12559,16206,17408,27560,1,80128,39945,13459,5,1,26,2,4,1,27,3,4,2,0,4
341423,7.864627,2,3,2,3,1,23824,45,8712dd3ce2e36f2fbcd16b24aa3b89bd1c5203a39e578b...,0,18,2,1,2,1,2.0,8.0,7,5,0,0,0,1,1,2,3,1,1,1.0,1.0,1,1,1.0,1.0,2,2,2.0,2.0,0.200000,0.500000,0.200000,0.400000,1,1,1.0,1.0,3,3,3.0,3.0,...,2.242163,2,1,126,42.941800,27,2,25,4.138824,3,1,28,3.287663,3,1,11,3.108781,3,1,31618,220256,36861,199403,53604,382100,182950,123267,53604,162447,181013,12648,17276,15858,27560,1,70994,45307,27248,5,1,27,3,4,2,0,4,4,2,1,5


In [41]:
train_X = train_df.drop(['reservation_id', 'memberid','amount_spent_per_room_night_scaled'],axis=1)
test_X = test_df.drop(['reservation_id', 'memberid', 'amount_spent_per_room_night_scaled'],axis=1)
y = train_df['amount_spent_per_room_night_scaled']
train_y = y

X = train_X

In [42]:
def runLGB(train_X, train_y, test_X, test_y=None, test_X2=None): 
    params = {}
    params["objective"] = "regression"
    params['metric'] = 'rmse'
    params["max_depth"] = 8
    params["min_data_in_leaf"] = 1
    params["learning_rate"] = 0.01
    params["bagging_fraction"] = 0.7
    params["feature_fraction"] = 0.7
    params["bagging_freq"] = 5
    params["bagging_seed"] = 0
    params["verbosity"] = -1
    num_rounds = 20000

    plst = list(params.items())
    lgtrain = lgb.Dataset(train_X, label=train_y)

    if test_y is not None:
        lgtest = lgb.Dataset(test_X, label=test_y)
        model = lgb.train(params, lgtrain,  num_rounds, valid_sets=[lgtest], 
                          early_stopping_rounds=100, verbose_eval=500)
    else:
        lgtest = lgb.Dataset(test_X)
        model = lgb.train(params, lgtrain,   num_rounds)

    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_test_y2 = model.predict(test_X2, num_iteration=model.best_iteration)

    loss = 0
    if test_y is not None:
        loss = np.sqrt(metrics.mean_squared_error(test_y, pred_test_y))
        print(loss)
        return pred_test_y, loss, pred_test_y2, model
    else:
        return pred_test_y, loss, pred_test_y2, model

In [43]:
cv_scores = []
pred_test_full = 0
kf = model_selection.GroupKFold(n_splits=5)
for dev_index, val_index in kf.split(X, y, train_df["reservation_id"].values):
#     print(train_df["reservation_id"].loc[val_index].unique())
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    pred_val, loss, pred_test,model = runLGB(dev_X, dev_y, val_X, val_y, test_X)
    pred_test_full += pred_test
    cv_scores.append(loss)
    print(cv_scores)
pred_test_full /= 5.
print(sum(cv_scores)/5)

Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 0.975721
[1000]	valid_0's rmse: 0.96994
[1500]	valid_0's rmse: 0.96866
[2000]	valid_0's rmse: 0.968022
[2500]	valid_0's rmse: 0.96768
Early stopping, best iteration is:
[2406]	valid_0's rmse: 0.967667
0.967666670680529
[0.967666670680529]
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 0.974164
[1000]	valid_0's rmse: 0.968694
[1500]	valid_0's rmse: 0.967344
[2000]	valid_0's rmse: 0.966674
[2500]	valid_0's rmse: 0.966371
[3000]	valid_0's rmse: 0.966093
[3500]	valid_0's rmse: 0.965834
[4000]	valid_0's rmse: 0.965577
Early stopping, best iteration is:
[4269]	valid_0's rmse: 0.965476
0.9654762771207941
[0.967666670680529, 0.9654762771207941]
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 0.980509
[1000]	valid_0's rmse: 0.975036
[1500]	valid_0's rmse: 0.973712
[2000]	valid_0's rmse: 0.973072
[2500]	valid_0's rmse: 0.972682
Early stop

In [44]:
      
# # Fill the is_pass variable with the predictions
submission_df['amount_spent_per_room_night_scaled']= pd.DataFrame(pred_test_full)

# # Converting the submission file to csv format
submission_df.to_csv('5fold_v1_lgb_1.csv', index=False)